# 開発用ノートブック1

In [69]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

必要な機能は
- notionDBからテキストを取得
- テキストをopenAIへ送信、返信を受け取る
- 受け取ったテキストをnotionDBに保存

*大きく分けて二つのサービスをAPIを通じて接続する。まずはローカルで実行できる形にする。*
## notion関連
テキストの取得

In [70]:

# .envファイルから環境変数を読み込む
load_dotenv()

# Notion API
notion_api_key = os.getenv("NOTION_TOKEN")
notion_db_input_id = os.getenv("NOTION_DB_INPUT")

if not notion_api_key or not notion_db_input_id:
    raise ValueError("環境変数 NOTION_TOKEN または NOTION_DB_INPUT が設定されていません")

# config header
headers = {
    "Authorization": f"Bearer {notion_api_key}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

try:
    # データベースをクエリするリクエスト
    response = requests.post(
        f"https://api.notion.com/v1/databases/{notion_db_input_id}/query",
        headers=headers
    )
    
    # レスポンスを取得
    response.raise_for_status()  # エラーチェック
    data = response.json()
    
    # 各ページのIDを取得
    results = data.get('results', [])  
    if not results:
        print("データベースにページが存在しません")
    else:
        for result in results:
            page_id = result.get('id')
            print(f"Page ID: {page_id}")

except requests.exceptions.RequestException as e:
    print(f"APIリクエストエラー: {e}")
except Exception as e:
    print(f"予期せぬエラー: {e}")

Page ID: 1d256d7a-44ed-8051-82d6-dec635dd92b9
Page ID: 1d256d7a-44ed-808d-b487-f86440f9308a
Page ID: 1d256d7a-44ed-80bb-80d5-e4f7e382c737
Page ID: 1d256d7a-44ed-8091-b45a-d688cb43c460
Page ID: 1d256d7a-44ed-8015-9993-f5b50b94e6ca
Page ID: 1cb56d7a-44ed-80ab-acfc-d2ec7e52976e
Page ID: 1c956d7a-44ed-80b2-a183-e00f465afb3f


In [71]:
print(result.get('properties').get('タイトル'))

進捗レポート


In [81]:
def Get_Database_Items_Id(Notion_API_Key, Database_Id, X_Days_Ago=None):


    # ヘッダーを設定
    headers = {
        "Authorization": f"Bearer {Notion_API_Key}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }

    try:
        # データベースをクエリするリクエスト
        response = requests.post(
            f"https://api.notion.com/v1/databases/{Database_Id}/query",
            headers=headers
        )
        
        # レスポンスを取得
        response.raise_for_status()  # エラーチェック
        data = response.json()
        
        # 各ページの情報を辞書に追加
        results = data.get('results', [])  
        if not results:
            print("データベースにページが存在しません")
            return pd.DataFrame()
        else:
            # 複数のページの情報を格納するためにリストを使用
            pages = []
            
            for result in results:
                
                page_info = {
                    'title': result.get('properties').get('タイトル').get('title')[0].get('plain_text'),
                    'page_id': result.get('id'),
                    'created_time': result.get('created_time')
                }
                pages.append(page_info)

    except requests.exceptions.RequestException as e:
        print(f"APIリクエストエラー: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"予期せぬエラー: {e}")
        return pd.DataFrame()

    # DataFrameを作成
    df = pd.DataFrame(pages)

    # 日付でフィルタリング
    if X_Days_Ago is not None:
        current_date = pd.Timestamp.now(tz='UTC')
        x_days_ago = current_date - pd.Timedelta(days=X_Days_Ago)
        df = df[pd.to_datetime(df['created_time']).between(x_days_ago, current_date)]

    return df

In [92]:
from notion_utils import Get_Database_Items_Id_test
content = Get_Database_Items_Id_test(notion_api_key, notion_db_input_id, 6)
display(content)

,title,page_id,created_time
0,進捗レポート,1d256d7a-44ed-8051-82d6-dec635dd92b9,2025-04-11T04:12:00.000Z
1,進捗レポート,1d256d7a-44ed-808d-b487-f86440f9308a,2025-04-11T04:10:00.000Z
2,進捗レポート,1d256d7a-44ed-80bb-80d5-e4f7e382c737,2025-04-11T04:10:00.000Z
3,進捗レポート,1d256d7a-44ed-8091-b45a-d688cb43c460,2025-04-11T04:09:00.000Z
4,進捗レポート,1d256d7a-44ed-8015-9993-f5b50b94e6ca,2025-04-11T04:06:00.000Z
